<a href="https://colab.research.google.com/github/Onethybeing/pytorch_implementation/blob/main/simpleregression_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install kaggle

# 2. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 3. Set up Kaggle credentials
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# 4. Download the specific insurance dataset
!kaggle datasets download mirichoi0218/insurance

# 5. Unzip the downloaded files (change the filename if needed)
!unzip insurance.zip -d /content/insurance_data


Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/mirichoi0218/insurance
License(s): DbCL-1.0
  0% 0.00/16.0k [00:00<?, ?B/s]
100% 16.0k/16.0k [00:00<00:00, 46.6MB/s]
Archive:  insurance.zip
  inflating: /content/insurance_data/insurance.csv  


In [3]:
import os
print(os.getcwd())

/content


In [10]:
print(os.listdir('/content/insurance_data'))

['insurance.csv']


In [12]:
os.listdir('/content')

['.config',
 'drive',
 'insurance.zip',
 'kaggle.json',
 '.ipynb_checkpoints',
 'insurance_data',
 'sample_data']

In [15]:
import pandas as pd
df = pd.read_csv('/content/insurance_data/insurance.csv')

In [16]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [18]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [24]:
train_df,test_df = train_test_split(df,test_size = 0.2,random_state = 42)

In [70]:
#encode cat variable
label_encoder = {}
cat_columns = ['sex','smoker','region']
for col in cat_columns:
  le = LabelEncoder()
  train_df[col] =  le.fit_transform(train_df[col])
  test_df[col] = le.transform(test_df[col])
  label_encoder[col] = le

In [71]:
#feature and  target
x_train = train_df.drop('charges',axis =1)
y_train = train_df['charges']
x_test = test_df.drop('charges',axis =1)
y_test = test_df['charges']

In [72]:
x_train.head()

,age,sex,bmi,children,smoker,region
560,46,0,19.95,2,0,1
1285,47,0,24.32,0,0,0
1142,52,0,24.86,0,0,2
969,39,0,34.32,5,0,2
486,54,0,21.47,3,0,1


In [73]:
x_test.shape

(268, 6)

In [74]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [75]:
x_train

array([[ 0.47222651, -1.0246016 , -1.75652513,  0.73433626, -0.50874702,
        -0.45611589],
       [ 0.54331294, -1.0246016 , -1.03308239, -0.91119211, -0.50874702,
        -1.35325561],
       [ 0.8987451 , -1.0246016 , -0.94368672, -0.91119211, -0.50874702,
         0.44102382],
       ...,
       [ 1.3252637 ,  0.97598911, -0.89153925, -0.91119211, -0.50874702,
        -1.35325561],
       [-0.16755139, -1.0246016 ,  2.82086429,  0.73433626,  1.96561348,
         1.33816354],
       [ 1.1120044 ,  0.97598911, -0.10932713, -0.91119211, -0.50874702,
         1.33816354]])

In [76]:
x_train_tensor =torch.tensor(x_train,dtype=torch.float32)
x_test_tensor =torch.tensor(x_test,dtype=torch.float32)
y_train_tensor =torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
y_test_tensor =torch.tensor(y_test.values,dtype=torch.float32).view(-1,1)

In [77]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
x_train_tensor, y_train_tensor = x_train_tensor.to(device), y_train_tensor.to(device)
x_test_tensor, y_test_tensor = x_test_tensor.to(device), y_test_tensor.to(device)


In [78]:
y_train_tensor.shape

torch.Size([1070, 1])

In [90]:
class SimpleNNRegressionModel(nn.Module):
  def __init__(self,input_dim,output_dim):
    super(SimpleNNRegressionModel,self).__init__()
    self.network = nn.Sequential(
        nn.Linear(6,64),
        nn.ReLU(),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Linear(128,1)
    )
  def forward(self,x):
    return self.network(x)

In [91]:
model = SimpleNNRegressionModel(input_dim=1,output_dim =1)
criteria = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [98]:
epochs = 1000

for epoch in range(epochs):
    model.train()  # set to training mode

    # Forward pass
    outputs = model(x_train_tensor)
    loss = criteria(outputs, y_train_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Logging every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    predictions = model(x_test_tensor)
    test_loss = criteria(predictions, y_test_tensor)
    print(f"\nTest MSE: {test_loss.item():.4f}")


Epoch [10/1000], Loss: 23055334.0000
Epoch [20/1000], Loss: 23028270.0000
Epoch [30/1000], Loss: 23000666.0000
Epoch [40/1000], Loss: 22973730.0000
Epoch [50/1000], Loss: 22947624.0000
Epoch [60/1000], Loss: 22922328.0000
Epoch [70/1000], Loss: 22897182.0000
Epoch [80/1000], Loss: 22871368.0000
Epoch [90/1000], Loss: 22846340.0000
Epoch [100/1000], Loss: 22822874.0000
Epoch [110/1000], Loss: 22799872.0000
Epoch [120/1000], Loss: 22777358.0000
Epoch [130/1000], Loss: 22755174.0000
Epoch [140/1000], Loss: 22733624.0000
Epoch [150/1000], Loss: 22712298.0000
Epoch [160/1000], Loss: 22691646.0000
Epoch [170/1000], Loss: 22671294.0000
Epoch [180/1000], Loss: 22651566.0000
Epoch [190/1000], Loss: 22632442.0000
Epoch [200/1000], Loss: 22613806.0000
Epoch [210/1000], Loss: 22594912.0000
Epoch [220/1000], Loss: 22575660.0000
Epoch [230/1000], Loss: 22556698.0000
Epoch [240/1000], Loss: 22538064.0000
Epoch [250/1000], Loss: 22518272.0000
Epoch [260/1000], Loss: 22497042.0000
Epoch [270/1000], Los

In [102]:
y_pred = predictions.detach().numpy()

In [110]:
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
y_test_numpy = y_test_tensor.numpy()
# metrics calculation
mse = mean_squared_error(y_test_numpy,y_pred)
mae = mean_absolute_error(y_test_numpy,y_pred)
r2 = r2_score(y_test_numpy,y_pred)

In [111]:
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R2 Score: {r2:.4f}")

MSE: 22450390.0000
MAE: 2874.4399
R2 Score: 0.8554


In [115]:
def predict_charges(age,sex,bmi,children,smoker,region):
  input_data = pd.DataFrame([[age,sex,bmi,children,smoker,region]],columns = ['age','sex','bmi','children','smoker','region'])
  for col in ['sex','smoker','region']:
    ls = LabelEncoder()
    train_df[col] = ls.fit_transform()